In [7]:
video_path = "Kobe.mp4"
# Constructor allocates memory and a threaded decoder
# instance per video. At the moment it takes two arguments:
# path to the video file, and a wanted stream.
reader = torchvision.io.VideoReader(video_path, "video")

# The information about the video can be retrieved using the
# `get_metadata()` method. It returns a dictionary for every stream, with
# duration and other relevant metadata (often frame rate)
reader_md = reader.get_metadata()

# metadata is structured as a dict of dicts with following structure
# {"stream_type": {"attribute": [attribute per stream]}}
#
# following would print out the list of frame rates for every present video stream
print(reader_md["video"]["fps"])

# we explicitly select the stream we would like to operate on. In
# the constructor we select a default video stream, but
# in practice, we can set whichever stream we would like
video.set_current_stream("video:0")

RuntimeError: Not compiled with video_reader support, to enable video_reader support, please install ffmpeg (version 4.2 is currently supported) andbuild torchvision from source.

In [15]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torchvision 
import torch


images = torchvision.io.read_video('Kobe.mp4', end_pts=10, pts_unit='sec') 
print(images)
images = images[0].numpy()


i = cv2.blur(images[0], (7,7))
cv2.imshow("Image", i)
cv2.waitKey(0)
cv2.destroyAllWindows()

images = np.concatenate((images, i[np.newaxis]), axis=0)

(tensor([[[[ 39,  15,  33],
          [ 32,   8,  26],
          [ 33,   9,  27],
          ...,
          [  9,   4,   6],
          [  8,   3,   5],
          [  8,   3,   5]],

         [[ 45,  21,  39],
          [ 37,  13,  31],
          [ 34,  10,  28],
          ...,
          [  8,   3,   5],
          [  8,   3,   5],
          [  8,   3,   5]],

         [[ 58,  34,  52],
          [ 45,  21,  39],
          [ 37,  13,  31],
          ...,
          [  8,   1,   4],
          [  9,   2,   5],
          [  9,   2,   5]],

         ...,

         [[ 61,  52,  76],
          [ 60,  51,  75],
          [ 57,  48,  72],
          ...,
          [ 70,  62,  79],
          [ 21,  13,  30],
          [  0,   0,   5]],

         [[ 67,  58,  82],
          [ 65,  56,  80],
          [ 62,  53,  77],
          ...,
          [ 68,  60,  77],
          [ 25,  17,  34],
          [  5,   0,  14]],

         [[ 58,  49,  73],
          [ 56,  47,  71],
          [ 54,  45,  69],
        

In [16]:
images.shape

(301, 720, 1280, 3)

In [17]:
images = cv2.convertScaleAbs(images) 
for i in range(images.shape[0]):
    #print(images[i].shape)
    cv2.imshow("Image"+str(i + 1), images[i])
    if cv2.waitKey(0) == ord('q'):
        break
    cv2.destroyAllWindows()
cv2.destroyAllWindows()

In [18]:

batch = images                       # trasformazione di batch di immagini normali(batch) in batch di immagini grayscale
images = np.ones((1,batch.shape[1], batch.shape[2]))                # images perchè cvtColor non accetta batch
for img in batch:
    #print(img.shape)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gray.reshape((1, batch.shape[1],batch.shape[2]))
    
    images = np.concatenate((images, gray), axis = (0))
    
images = images[1:]

In [19]:
images = cv2.convertScaleAbs(images) 
for i in range(images.shape[0]):
    cv2.imshow("Image", images[i])
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()


images = images[:,np.newaxis]

images.shape


(301, 1, 720, 1280)

In [20]:

def convolution(input, kernel):
    
    if(torch.cuda.is_available()):
        input = input.cuda()
        kernel = kernel.cuda()
        print('CUDA activated! ' + 'input : ' + str(input.device) + '  kernel  '  + str(kernel.device))
    
    n, iC, H, W = input.shape
    kH = kernel.shape[2]
    kW = kernel.shape[3]
    oC = kernel.shape[0]
    
    out = torch.zeros((n, oC, (H - kH + 1), (W - kW + 1)))
    if(torch.cuda.is_available()):
        out = out.cuda()
        print('CUDA activated out! ' + '  out : ' + str(out.device))
    for r in range(0, H - kH + 1):
        for c in range(0, W - kW + 1):
            out[:,:, r, c] = (input[:,:,r:r+kH,c:c+kW].reshape((n, 1, iC, kH, kW)) * kernel.reshape((1, oC, iC, kH, kW))).sum((2, 3, 4))
            #print(res.shape, (r, c))
    return out

In [21]:
i = int(input('which kernel ? : '))
if i == 1:
    kernel = np.array([[0, -1, 0],
                  [-1, 4, -1], 
                  [0, -1, 0]], dtype=np.float32)
else:
    kernel = np.array([[-1, -1, -1],
                  [-1, 8, -1], 
                  [-1, -1, -1]], dtype=np.float32)
    
kernel



which kernel ? : 1


array([[ 0., -1.,  0.],
       [-1.,  4., -1.],
       [ 0., -1.,  0.]], dtype=float32)

In [22]:
k  = kernel + np.zeros((3, 3))
k = k[np.newaxis,np.newaxis]

k.shape

(1, 1, 3, 3)

In [23]:
o = convolution(torch.from_numpy(images), torch.from_numpy(k))
o.shape

CUDA activated! input : cuda:0  kernel  cuda:0
CUDA activated out!   out : cuda:0


torch.Size([301, 1, 718, 1278])

In [71]:
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.get_device_name(0)
torch.cuda.memory_reserved()
type(o)

torch.Tensor

In [74]:
if torch.cuda.is_available():    
    print('Cuda available ' + str(o.device))
    o = o.cpu().numpy()
else:
    o = o.numpy()

o = np.squeeze(o)
o = cv2.convertScaleAbs(o) 
for i in range(o.shape[0]):
    cv2.imshow("Image"+str(i), o[i])
    if cv2.waitKey(0) == ord('q'):
        break
    cv2.destroyAllWindows()
cv2.destroyAllWindows()
    
vars = np.var(o, axis=(1, 2))

mask = vars >= 140

AttributeError: 'numpy.ndarray' object has no attribute 'device'

In [75]:
m = o.copy()

In [76]:
t = ['Blurry', 'Not Blurry']
for i in range(m.shape[0]):
    if vars[i] < 134:
        text = t[0]
    else:
        text = t[1]
    r = cv2.putText(m[i], "{}: {:.2f}".format(text, vars[i]), (0,90),cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3, bottomLeftOrigin=False,)
    cv2.imshow("Image"+str(i + 1), r)
    if cv2.waitKey(0) == ord('q'):
        break
    cv2.destroyAllWindows()
cv2.destroyAllWindows()


In [77]:
m[mask].shape

(300, 718, 1278)